In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer, util
import warnings

# Suppress warnings
warnings.filterwarnings('ignore')

# ======================
# CONFIGURATION
# ======================

# Display settings
pd.set_option('display.max_colwidth', 50)
pd.set_option('display.width', 1000)
plt.style.use('ggplot')

# ======================
# SAMPLE DATA
# ======================

samples = [
    {
        "question": "Explain photosynthesis",
        "reference": "Photosynthesis converts light energy to chemical energy in plants.",
        "student": "Plants make food from sunlight in chloroplasts through photosynthesis."
    },
    {
        "question": "Newton's second law",
        "reference": "Force equals mass times acceleration (F=ma).",
        "student": "F=ma. I got this from ChatGPT."
    },
    {
        "question": "Blockchain technology",
        "reference": "Decentralized, secure transaction recording system.",
        "student": "A blockchain securely records transactions across many computers."
    },
    {
        "question": "Deep learning",
        "reference": "Uses multi-layer neural networks for pattern recognition.",
        "student": "Deep learning uses layered neural networks. ChatGPT explained this."
    }
]

# ======================
# CORE FUNCTIONS
# ======================

def load_model():
    """Load the sentence transformer model"""
    return SentenceTransformer('all-MiniLM-L6-v2')

def calculate_similarity(text1, text2, model):
    """Calculate semantic similarity between two texts"""
    emb1 = model.encode(text1, convert_to_tensor=True)
    emb2 = model.encode(text2, convert_to_tensor=True)
    return util.pytorch_cos_sim(emb1, emb2).item()

def detect_ai(answer):
    """Simulate AI detection (replace with actual API call if needed)"""
    answer_lower = answer.lower()
    ai_keywords = ['chatgpt', 'ai', 'generated', 'model']
    
    if any(keyword in answer_lower for keyword in ai_keywords):
        return "AI-Generated (simulated)", 0.85
    return "Human-Written (simulated)", 0.75

def determine_verdict(ai_result, similarity):
    """Determine final verdict based on detection and similarity"""
    if "AI" in ai_result:
        if similarity < 0.6:
            return "❌ AI (Low Relevance)"
        elif similarity > 0.75:
            return "⚠️ AI (High Similarity)"
        return "⚠️ AI (Medium Similarity)"
    else:
        if similarity > 0.85:
            return "✅ Human (Excellent)"
        return "✅ Human (Good)"

def analyze_answers(samples, model):
    """Analyze all answers and return DataFrame"""
    results = []
    
    for sample in samples:
        similarity = calculate_similarity(
            sample["reference"], 
            sample["student"], 
            model
        )
        
        ai_detection, confidence = detect_ai(sample["student"])
        verdict = determine_verdict(ai_detection, similarity)
        
        results.append({
            "Question": sample["question"],
            "Answer": sample["student"],
            "Similarity": round(similarity, 3),
            "AI Detection": ai_detection,
            "Confidence": confidence,
            "Verdict": verdict
        })
    
    return pd.DataFrame(results)

def create_visualizations(df):
    """Generate analysis visualizations"""
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    # Similarity Scores
    df['Similarity'].plot(kind='bar', ax=axes[0], color='skyblue', alpha=0.7)
    axes[0].set_title('Answer Similarity Scores')
    axes[0].set_ylabel('Similarity (0-1)')
    axes[0].set_ylim(0, 1)
    
    # Verdict Distribution
    verdict_counts = df['Verdict'].value_counts()
    verdict_counts.plot(kind='pie', ax=axes[1], autopct='%1.1f%%', 
                      colors=['#ff9999','#66b3ff','#99ff99','#ffcc99'])
    axes[1].set_title('Verdict Distribution')
    axes[1].set_ylabel('')
    
    # Confidence Scores
    df['Confidence'].plot(kind='barh', ax=axes[2], color='lightgreen')
    axes[2].set_title('Detection Confidence')
    axes[2].set_xlim(0, 1)
    
    plt.tight_layout()
    plt.show()

def print_report(df):
    """Print formatted report"""
    print("\n" + "="*80)
    print("COMPREHENSIVE ANSWER ANALYSIS REPORT".center(80))
    print("="*80 + "\n")
    display(df)
    
    print("\nSUMMARY STATISTICS:")
    print(f"Average Similarity: {df['Similarity'].mean():.2f}")
    print(f"AI Detection Rate: {(df['AI Detection'].str.contains('AI')).mean():.1%}")
    print(f"Average Confidence: {df['Confidence'].mean():.1%}")

# ======================
# MAIN EXECUTION
# ======================

if __name__ == "__main__":
    # Load ML model
    model = load_model()
    
    # Analyze answers
    results_df = analyze_answers(samples, model)
    
    # Generate report
    print_report(results_df)
    
    # Create visualizations
    create_visualizations(results_df)

In [ ]:
'''What We're Building: An "Answer Checker" Tool
This program checks student answers against reference answers to:

Detect if answers were written by AI

See how similar they are to the correct answer

Give a final grade/verdict

How It Works (Step by Step):
Setting Up the Toolbox

We import ready-made tools (like a calculator for text similarity)

Set up how we want the results to look (tables and charts)

Sample Questions & Answers

We provide some example questions

For each, we have:

The "perfect" reference answer

A student's answer (some written by humans, some mentioning ChatGPT)

The Checking Process
For each student answer:

Similarity Check:

Compares how close the student answer is to the reference

Gives a score from 0 (completely different) to 1 (identical meaning)

AI Detection:

Looks for clues like "ChatGPT" in the answer

(In a real version, this would use an AI detection API)

Final Verdict:

Combines both checks to give ❌/⚠️/✅ grades

Showing the Results

A clean table showing all answers and their scores

Three helpful charts:

Bar chart of similarity scores

Pie chart of verdict percentages

Confidence level of our AI detection

Summary Statistics

Average similarity across all answers

Percentage of answers flagged as AI

Overall confidence in our checks

Example Walkthrough:
For the question "What is Newton's second law?"

Reference: "Force equals mass times acceleration (F=ma)."

Student answer: "F=ma. I got this from ChatGPT."

The program:

Sees "ChatGPT" → flags as AI-generated

Calculates 0.48 similarity (moderately close but not exact)

Gives verdict: "⚠️ AI (Medium Similarity)"

Key Things to Note:
This is a simulation (we fake the AI detection part)

In real use, you'd connect to an actual AI detection service

The similarity scoring is real and uses advanced text comparison

All the visualizations update automatically

Why This Is Useful:
Teachers can quickly check many answers

Helps identify potential cheating

Shows which answers are most/least accurate

All results are visual and easy to understand'''